# 如何使用时间加权向量存储检索器

此[检索器](/docs/concepts/retrievers/)结合了语义[相似度](/docs/concepts/embedding_models/#measure-similarity)和时间衰减。

其评分算法为：

```
semantic_similarity + (1.0 - decay_rate) ^ hours_passed
```

特别地，`hours_passed` 指的是自检索器中的对象**上次被访问**以来经过的小时数，而不是自其创建以来经过的小时数。这意味着经常被访问的对象会保持“新鲜”状态。

In [1]:
from datetime import datetime, timedelta

import faiss
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain_community.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

## 低衰减率

较低的 `decay rate`（在此，为了极端化，我们将它设置为接近 0）意味着记忆会被“记住”更长时间。`decay rate` 为 0 意味着记忆永远不会被遗忘，这使得该检索器等同于向量查找。

In [2]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})
retriever = TimeWeightedVectorStoreRetriever(
    vectorstore=vectorstore, decay_rate=0.0000000000000000000000001, k=1
)

In [3]:
yesterday = datetime.now() - timedelta(days=1)
retriever.add_documents(
    [Document(page_content="hello world", metadata={"last_accessed_at": yesterday})]
)
retriever.add_documents([Document(page_content="hello foo")])

['73679bc9-d425-49c2-9d74-de6356c73489']

In [4]:
# "Hello World" is returned first because it is most salient, and the decay rate is close to 0., meaning it's still recent enough
retriever.invoke("hello world")

[Document(metadata={'last_accessed_at': datetime.datetime(2024, 10, 22, 16, 37, 40, 818583), 'created_at': datetime.datetime(2024, 10, 22, 16, 37, 37, 975074), 'buffer_idx': 0}, page_content='hello world')]

## 高衰减率

当 `decay rate` 较高时（例如有多个 9），`recency score` 会迅速变为 0！ 如果将其设置为 1，`recency` 对所有对象都将为 0，再次使其等同于向量查找。

In [5]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})
retriever = TimeWeightedVectorStoreRetriever(
    vectorstore=vectorstore, decay_rate=0.999, k=1
)

In [6]:
yesterday = datetime.now() - timedelta(days=1)
retriever.add_documents(
    [Document(page_content="hello world", metadata={"last_accessed_at": yesterday})]
)
retriever.add_documents([Document(page_content="hello foo")])

['379631f0-42c2-4773-8cc2-d36201e1e610']

In [7]:
# "Hello Foo" is returned first because "hello world" is mostly forgotten
retriever.invoke("hello world")

[Document(metadata={'last_accessed_at': datetime.datetime(2024, 10, 22, 16, 37, 46, 553633), 'created_at': datetime.datetime(2024, 10, 22, 16, 37, 43, 927429), 'buffer_idx': 1}, page_content='hello foo')]

## 虚拟时间

您可以利用 LangChain 中的一些工具来模拟时间组件。

In [8]:
from langchain_core.utils import mock_now

In [9]:
# Notice the last access time is that date time

tomorrow = datetime.now() + timedelta(days=1)

with mock_now(tomorrow):
    print(retriever.invoke("hello world"))

[Document(metadata={'last_accessed_at': MockDateTime(2024, 10, 23, 16, 38, 19, 66711), 'created_at': datetime.datetime(2024, 10, 22, 16, 37, 43, 599877), 'buffer_idx': 0}, page_content='hello world')]
